# Reasearch Track II Assignment 2

---

**Author's Name: Omotoye Shamsudeen Adekoya** <br>
**Student ID: 5066348**

---

## NoteBook Description

This Notebook has two functions; it serves as a user interface for the control of non-holonomic robot in a 3D environment and it also show a graphical feedback of the robot state and behaviour. 

In [1]:
import ipywidgets as widgets
from ipywidgets import Button, Layout, HBox, VBox
from ipywidgets import Button, GridBox, Layout, ButtonStyle
%matplotlib widget


## Robot Control

In [23]:
import matplotlib.pyplot as plt
import rospy
import tf
from nav_msgs.msg import Odometry
import numpy as np
from matplotlib.animation import FuncAnimation
from tf.transformations import euler_from_quaternion
from math import *
from matplotlib.widgets import Button as Btn
from matplotlib.lines import Line2D 
from matplotlib.patches import Arrow, Circle
from tf.transformations import euler_from_quaternion
from math import sin, cos


class Visualiser:
    def __init__(self):
        self.fig = plt.figure(figsize=(8,6))
        self.ax = self.fig.add_axes([0.48,0.17,0.5,0.67])
        self.info_ax = self.fig.add_axes([0.05,0.05, 0.35,0.9])
        self.info_ax.grid(False)
        self.info_ax.set_facecolor('#faebd7')
#         self.info_ax.tick_params(left = False, right = False , labelleft = False ,
#                 labelbottom = False, bottom = False)
        self.btax = self.fig.add_axes([0.8, 0.02, 0.2, 0.05])
        self.line = Line2D([],[], color='g', linestyle='dashed', marker='.',
     markerfacecolor='blue')
        self.ax.add_line(self.line)
        self.arrow = Arrow(0, 0, 0, 0,)
        self.ax.add_patch(self.arrow)
        self.circle = Circle((0, 0), radius=0)
        self.ax.add_patch(self.circle)
        self.target = Circle((0, 0), radius=0)
        self.ax.add_patch(self.target)
        self.ax.set_title('Robot Position and Goal Visualizer')
        self.ax.set_xlabel('X Coordinate')
        self.ax.set_ylabel('Y Coordinate')
        self.x_data, self.y_data = [], []
        self.pose_data = 'X: 0.0\n\nY: 0.0\n\nOrientation: 0.0'
        self.vel_data = 'Linear Velocity: 0.0\n\nAngular Velocity: 0.0'
        self.goal_data = 'Target:\n    x: 0.0\n    y: 0.0\n\nDistance to Target: 0.0'
        self.info_ax.text(0.1, 0.92, 'POSITION', style='oblique', fontsize=16,
        bbox={'facecolor': 'red','alpha': 0.5, 'pad': 10})
        self.info_ax.text(0.1, 0.7, self.pose_data, style='normal', fontsize=12,
        bbox={'facecolor': 'red','alpha': 0,'pad': 10})
        self.info_ax.text(0.1, 0.60, 'VELOCITIES', style='oblique', fontsize=16,
        bbox={'facecolor': 'red','alpha': 0.5, 'pad': 10})
        self.info_ax.text(0.1, 0.25, 'GOAL INFO', style='oblique', fontsize=16,
        bbox={'facecolor': 'red','alpha': 0.5, 'pad': 10})
        self.info_ax.text(0.1, 0.45, self.vel_data, style='normal', fontsize=12,
        bbox={'facecolor': 'red','alpha': 0,'pad': 10})
        self.info_ax.text(0.1, 0.02, self.goal_data, style='normal', fontsize=12,
        bbox={'facecolor': 'red','alpha': 0,'pad': 10})
        
        
        
    def plot_init(self):
        self.ax.set_xlim(-6, 6)
        self.ax.set_ylim(-6, 6)
        return self.line
    
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        rotation = msg.pose.pose.orientation
        quaternion = [rotation.x, rotation.y, rotation.z, rotation.w]
        # Using tuple unpacking to get the roll, pitch and yaw values for the euler tuple
        (roll, pitch, self.yaw) = euler_from_quaternion(quaternion)
        
    def update_plot(self, frame): 
        self.line.set_data(self.x_data, self.y_data)
        
        if len(self.x_data) != 0:
            self.arrow.remove()
            self.circle.remove()
            x = self.x_data[-1]
            y = self.y_data[-1]
            self.arrow = Arrow(x, y, (cos(self.yaw)), (sin(self.yaw)))
            self.ax.add_patch(self.arrow)
            self.circle = Circle((x, y), radius=0.4)
            self.ax.add_patch(self.circle)
        if rospy.has_param('/target_point'):
            target_point = rospy.get_param('/target_point')
            self.target.remove()
            self.target = Circle((target_point[0], target_point[1]), radius=0.2, color='r')
            self.ax.add_patch(self.target)
        return self.line
    
    def clear_path(self, event):
        self.x_data = []
        self.y_data = []
        

class AnalysisPlots:
    def __init__(self):
        self.reached_target = 0
        self.canceled_target = 0
        self.time_to_target = []
        self.fig = plt.figure(figsize=(8,8))
    
    def get_analysis_data(self):
        if (rospy.has_param('/reached_target')):
            self.reached_target = rospy.get_param('/reached_target')
        if (rospy.has_param('/canceled_target')):
            self.canceled_target = rospy.get_param('/canceled_target')
        if (rospy.has_param('/target_time')):
            self.time_to_target = rospy.get_param('/target_time')
            
    def init_barplot(self):
        self.axes_2 = self.fig.add_axes([0.1, 0.1, 0.7, 0.1])
        self.axbar = self.fig.add_axes([0.7, 0, 0.15, 0.05])
        self.axes_2.set_title('Number of Canceled and Reached Targets')
        self.get_analysis_data()
        self.axes_2.barh(('Reached\nTarget', 'Canceled\nTarget'),(self.reached_target, self.canceled_target),color=[ 'green','red'])
        
        
    def init_histplot(self):
        self.axes_1 = self.fig.add_axes([0.1, 0.4, 0.7, 0.5])
        self.axhist = self.fig.add_axes([0.7, 0.3, 0.15, 0.05])
        self.axes_1.set_title('Time to Reach Targets')
        self.axes_1.set_xlabel('Time to reach target')
        self.axes_1.set_ylabel('Frequency')
        self.get_analysis_data()
        self.axes_1.hist(self.time_to_target, stacked=True)
        

    def reload_bar(self, event):
        self.axes_2.cla()
        self.init_barplot()
    
    
    def reload_hist(self, event):
        self.axes_1.cla()
        self.init_histplot()
        



In [24]:
import rospy
import time
from rt2_assignment2.srv import Command, RandomPosition
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry 


from all_buttons import *

from ipywidgets import Button, Layout, HBox, VBox
plt.style.use('ggplot')

def start_simulation(a):
    ui_client("start")
    start_button.button_style = 'success'
    stop_button.button_style = ''
    
def stop_simulation(b):
    ui_client("stop")
    start_button.button_style = ''
    stop_button.button_style = 'danger'  
    
def move_forward(a):
    if (rospy.has_param('/linear_velocity')):
        lv_multiplier = rospy.get_param('/linear_velocity')
    else:
        lv_multiplier = 1
    vel = Twist()
    vel.linear.x = lv_multiplier*0.5
    pub.publish(vel)
    up_button.button_style = 'info'
    down_button.button_style = ''
    left_button.button_style = ''
    right_button.button_style = ''
    stop.button_style = ''
    
    
def move_backward(a):
    if (rospy.has_param('/linear_velocity')):
        lv_multiplier = rospy.get_param('/linear_velocity')
    else:
        lv_multiplier = 1
    vel = Twist()
    vel.linear.x = lv_multiplier*(-0.5)
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = 'info'
    left_button.button_style = ''
    right_button.button_style = ''
    stop.button_style = ''
  

def turn_left(a):
    if (rospy.has_param('/angular_velocity')):
        av_multiplier = rospy.get_param('/angular_velocity')
    else:
        av_multiplier = 1
    vel = Twist()
    vel.angular.z = av_multiplier*(-0.5)
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = ''
    left_button.button_style = 'info'
    right_button.button_style = ''
    stop.button_style = ''
   

def turn_right(a):
    if (rospy.has_param('/angular_velocity')):
        av_multiplier = rospy.get_param('/angular_velocity')
    else:
        av_multiplier = 1
    vel = Twist()
    vel.angular.z = av_multiplier*(0.5)
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = ''
    left_button.button_style = ''
    right_button.button_style = 'info'
    stop.button_style = ''

    
def stop_movement(a):
    vel = Twist()
    vel.linear.x = 0.0
    vel.angular.z = 0.0
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = ''
    left_button.button_style = ''
    right_button.button_style = ''
    stop.button_style = 'danger'
    
    
def on_value_change(change):
    x = change['new']
    rospy.set_param('/linear_velocity', x)
    
    
def on_valueang_change(change):
    z = change['new']
    rospy.set_param('/angular_velocity', z)


def button_event():
    start_button.on_click(start_simulation)
    stop_button.on_click(stop_simulation)
    up_button.on_click(move_forward)
    down_button.on_click(move_backward)
    left_button.on_click(turn_left)
    right_button.on_click(turn_right)
    stop.on_click(stop_movement)
    l_velSlider.observe(on_value_change, names='value')
    a_velSlider.observe(on_valueang_change, names='value')
    
def main():
    rospy.init_node('user_interface')
    button_event()


if __name__ == '__main__':
    rospy.wait_for_service('/user_interface')
    try:
        ui_client = rospy.ServiceProxy('/user_interface', Command)
    except rospy.ServiceException as e:
        print(f'Service call failed: {e}')
    main()
    vis = Visualiser()
    pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
    sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)

    sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
    ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
    
    clear_path = Btn(vis.btax, 'Clear Path')
    clear_path.on_clicked(vis.clear_path)
    plt.show(block=True)
    
VBox([l_velSlider, a_velSlider])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
HBox([start_button, stop_button])

In [5]:
display(control_pad)

GridBox(children=(Button(icon='arrow-up', layout=Layout(grid_area='up', height='50px', width='auto'), style=Bu…

In [6]:
anaplot = AnalysisPlots()
anaplot.init_barplot()
anaplot.init_histplot()

bnext = Btn(anaplot.axbar, 'Reload BarPlot')
anext = Btn(anaplot.axhist, 'Reload HistPlot')
bnext.on_clicked(anaplot.reload_bar)
anext.on_clicked(anaplot.reload_hist)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …